## Validate Azure ML SDK installation and get version number for debugging purposes

In [ ]:
# Check core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)

## Diagnostics
Opt-in diagnostics for better experience, quality, and security of future releases.

In [ ]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics = True)

## Initialize Workspace
Initialize a workspace object from persisted configuration.

In [ ]:
# Initialize Workspace
from azureml.core import Workspace

ws = Workspace.from_config()
print("Resource group: ", ws.resource_group)
print("Location: ", ws.location)
print("Workspace name: ", ws.name)

## Set experiment name and create project
Choose a name for your run history container in the workspace, and create a folder for the project.


In [ ]:
from os import path, makedirs
experiment_name = 'tensorboard-demo'

# experiment folder
exp_dir = '../projects/' + experiment_name

if not path.exists(exp_dir):
    makedirs(exp_dir)

# runs we started in this session, for the finale
runs = []

## Download Tensorflow TensorBoard demo code
Tensorflow's repository has an MNIST demo with extensive TensorBoard instrumentation. We'll use it here for our purposes.
Note that we don't need to make any code changes at all - the code works without modification from the Tensorflow repository.

In [ ]:
import requests
import os
import tempfile
tf_code = requests.get("https://raw.githubusercontent.com/tensorflow/tensorflow/r1.8/tensorflow/examples/tutorials/mnist/mnist_with_summaries.py")
with open(os.path.join(exp_dir, "mnist_with_summaries.py"), "w") as file:
    file.write(tf_code.text)

## Configure and run locally
We'll start by running this locally. While it might not initially seem that useful to use this for a local run - why not just run TB against the files generated locally? - even in this case there is some value to using this feature. Your local run will be registered in the run history, and your TensorBoard logs will be uploaded to the artifact store associated with this run. Later, you'll be able to restore the logs from any run, regardless of where it happened.

**Note** that for this run, you will need to install TensorFlow on your local machine by yourself. Further, the TensorBoard module (that is, the one included with TensorFlow) must be accessible to this notebook's kernel, as the local machine is what runs TensorBoard.

In [ ]:
from azureml.core.runconfig import RunConfiguration

# Create a run configuration.
run_config = RunConfiguration()
run_config.environment.python.user_managed_dependencies = True

# You can choose a specific Python environment by pointing to a Python path 
#run_config.environment.python.interpreter_path = '/home/ninghai/miniconda3/envs/sdk2/bin/python'

In [ ]:
from azureml.core import Experiment, Run
from azureml.core.script_run_config import ScriptRunConfig
import tensorflow as tf

logs_dir = os.path.join(os.curdir, "logs")
data_dir = os.path.abspath(os.path.join(os.curdir, "../MNIST_data"))

if not path.exists(data_dir):
    makedirs(data_dir)

os.environ["TEST_TMPDIR"] = data_dir

# Writing logs to ./logs results in their being uploaded to Artifact Service,
# and thus, made accessible to our TensorBoard instance.
arguments_list = ["--log_dir", logs_dir, "--data_dir", data_dir]

# Create an experiment
exp = Experiment(ws, experiment_name)

# If you would like the run to go for longer, add --max_steps 5000 to the arguments list:
# arguments_list += ["--max_steps", "5000"]

script = ScriptRunConfig(exp_dir,
                         script="mnist_with_summaries.py",
                         run_config=run_config,
                         arguments=arguments_list)

run = exp.submit(script)
runs.append(run)

In [ ]:
from azureml.train.widgets import RunDetails
RunDetails(run).show()

## Start TensorBoard
Now, while the run is in progress, we just need to start TensorBoard with the run as its target, and it will begin streaming logs.

In [ ]:
from azureml.contrib.tensorboard import Tensorboard

# The TensorBoard constructor takes an array of runs, so be sure and pass it in as a single-element array here
tb = Tensorboard([run])

# If successful, start() returns a string with the URI of the instance.
tb.start()

## Stop TensorBoard
When you're done, make sure to call the stop() method of the TensorBoard object, or it will stay running even after your job completes.

In [ ]:
tb.stop()

## Once more, with a Batch AI cluster
Just to prove we can, let's create a Batch AI cluster using MLC, and run our demo there, as well.

In [ ]:
from azureml.core.compute import BatchAiCompute
from azureml.core.compute_target import ComputeTargetException

compute_target_name = 'myazbai'

try:
    batch_ai_compute = BatchAiCompute(workspace=ws, name=compute_target_name)
    print('found existing:', batch_ai_compute.name)
except ComputeTargetException:
    print('creating new.')
    batch_ai_config = BatchAiCompute.provisioning_configuration(
        vm_size="Standard_NC6",
        vm_priority="dedicated",
        autoscale_enabled = True,
        cluster_min_nodes = 0,
        cluster_max_nodes = 4
    )
    batch_ai_compute = BatchAiCompute.create(
        ws, 
        name=compute_target_name, 
        provisioning_configuration=batch_ai_config
    )
    batch_ai_compute.wait_for_completion(show_output=True)

## Submit run using TensorFlow estimator
Again, we can use the TensorFlow estimator and everything is set up automatically.

In [ ]:
from azureml.train.dnn import TensorFlow

script_params = {
    "--log_dir": "./logs"
}

tf_estimator = TensorFlow(
    source_directory=exp_dir,
    compute_target=batch_ai_compute,
    entry_script='mnist_with_summaries.py',
    script_params=script_params
)

run = exp.submit(tf_estimator)

runs.append(run)

In [ ]:
from azureml.train.widgets import RunDetails
RunDetails(run).show()

## Start TensorBoard with this run
Once more...

In [ ]:
from azureml.contrib.tensorboard import Tensorboard

# The TensorBoard constructor takes an array of runs, so be sure and pass it in as a single-element array here
tb = Tensorboard([run])

# If successful, start() returns a string with the URI of the instance.
tb.start()

## Stop TensorBoard
When you're done, make sure to call the stop() method of the TensorBoard object, or it will stay running even after your job completes.

In [ ]:
tb.stop()

## Finale
If you've paid close attention, you'll have noticed that we've been saving the run objects in an array as we went along. We can start a TensorBoard instance that combines all of these run objects into a single process. This way, you can compare historical runs. You can even do this with live runs; if you made some of those previous runs longer via the --max_steps parameter, they might still be running, and you'll see them live in this instance as well.

In [ ]:
from azureml.contrib.tensorboard import Tensorboard

# The TensorBoard constructor takes an array of runs...
# and it turns out that we have been building one of those all along.
tb = Tensorboard(runs)

# If successful, start() returns a string with the URI of the instance.
tb.start()

## Stop TensorBoard
As you might already know, make sure to call the stop() method of the TensorBoard object, or it will stay running (until you kill the kernel associated with this notebook, at least).

In [ ]:
tb.stop()